<p><img src="imagenes/cabecera.png" width="900" align="center"></p>

# Trabajo práctico 3: Vectores de palabras

## Curso Procesamiento de Lenguaje Natural 

### Maestría en Tecnologías de la información



**Trabajo práctico porpuesto por:** Julio Waissman Vilanova (julio.waissman@unison.mx)

**Desarrollado por:** _Poner tu nombre y correo electrónico aquí_


En este trabajo práctico vamos a generar y usar un modelo de vectores de inserciones de palabras. Para este proyecto práctico, contamos con un corpus de preguntas que se han hecho en *Stackoverflow* para diferentes lenguajes de programación.

La idea final es contar con una función que, ante una frase, determine si es una pregunta sobre algun lenguaje de programación, que determine el lenguaje de programación, y que los de una recomendación sobre una pregunta similar. Dado que el corpus lo tengo solo en inglés (casi no hay preguntas técnicas en español), vamos a hacer todo en inglés para este ejercicio. Para la detectar si una frase es de interés nuestro, se utiliza un corpus de diálogos de peliculas de cine.

Si bien vamos a hacer todos los pasos juntos, este corpus ya es a escala real, y muchos de los problemas que se pueden tener en el desarrollo del trabajo práctico pueden ser de orden de memoria o de tiempo de procesamiento. Espero que se encuentren con algunos de esos problemas (aunque procuraré ahorrarselos) con el fin que obtengan una experiencia sobre el tipo de problemáticas que se tienen al desarrollar y usar modelos de vectores de palabras.


## 1. Obteniendo los documentos

Los documentos se encuentran en dos archivos *pickle*, los cuales los precargamos con los módulos necesarios para el trabajo práctico.

La llamada a todas las librerías que vas a necesitar, así como cargar los datos con los que vas a desarrollar la libreta, los dejo listos.

In [50]:
import warnings; warnings.simplefilter('ignore')
import re
import pickle
from collections import Counter
import numpy as np
import pandas as pd
import gensim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

!bunzip2 datos/preguntas_so.pkl.bz2
df_post = pd.read_pickle('datos/preguntas_so.pkl')
df_dialog = pd.read_pickle('datos/dialogos.pkl')
!bzip2 datos/preguntas_so.pkl

display(df_post.tail())
display(df_dialog.tail())

tags = Counter(df_post.loc[:,'tag'])
print("Las etiquetas (lenguajes) existentes son:")
print("{:>15s}{:>10}".format('Etiqueta', 'Entradas'))
for (tag, numero) in tags.most_common():
    print("{:>15s}{:10d}".format(tag, numero))

,post_id,title,tag
2171570,45887455,What is the difference between node.js and ayo...,javascript
2171571,45887857,Why do sequential containers have both size_ty...,c_cpp
2171572,45892983,"why 1 + + ""1"" === 2; +""1"" + + ""1"" === 2 and ""1...",javascript
2171573,45893693,Why does the first line work but the second li...,javascript
2171574,45898184,Can I safely convert struct of floats into flo...,c_cpp


,text,tag
218604,Lord Chelmsford seems to want me to stay back ...,dialogue
218605,I'm to take the Sikali with the main column to...,dialogue
218606,"Your orders, Mr Vereker?",dialogue
218607,"Good ones, yes, Mr Vereker. Gentlemen who can ...",dialogue
218608,Colonel Durnford... William Vereker. I hear yo...,dialogue


Las etiquetas (lenguajes) existentes son:
       Etiqueta  Entradas
             c#    394451
           java    383456
     javascript    375867
            php    321752
          c_cpp    281300
         python    208607
           ruby     99930
              r     36359
             vb     35044
          swift     34809


como podemos observar hay más de dos millones de documentos sobre consultas a *stack overflow* mientras que los diálogos de películas son aproximadamente 200 mil.

Lo que vamos a hacer a lo largo del proyecto es lo siguiente:

1. Normalizar y guardar cada una de las frases de ambos corpus y guardarlo en un archivo /texto plano/ (en unicode, por supuesto).

2. Utilizar FastText para obtener un modelo de palabras

3. Realizar una función que permita codificar frases con nuestro modelo

4. Desarrollar un clasificador que determine si una frase trata sobre posibles preguntas a /Stackoverflow/, o no.

5. Desarrollar un clasificador que determine el tema (lenguaje de programación) de una frase.

6. Desarrollar un recomendador que seleccione la pregunta más parecida a una frase.

7. Poner todo junto.

Por supuesto, para todas las etapas no es seguro que usar modelos de vectores de palabras sea la mejor opción, pero la idea de esta libreta es practicar semántica distribuida y sus aplicaciones.




## 2. Generando un modelo de palabras específico al problema

Para esto, lo priero es desarrollar un método de normalización de textos. 

**Genera un método de normalización de textos tal que:**
1. **Convierta a minúsculas todas las palabras.**
2. **Cambie los caracteres `[`, `]`, `(`, `)`, `{`, `}`, `|`, `/`, `@`, `;` y `,` por un espacio en blanco.**
3. **Elimine todos los caracteres que no sean alfabéticos, digitos, o los caracteres `#`, `+`, `_` o el espacio en blanco.**
4. **Elimina las palabras vacias.**

**Para esto, compila las expresiones regulares que uses antes, ya que eso ahorrará muchisimo tiempo más adelante. **

In [51]:
# AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES



# Función de normalización
def normaliza_texto(texto):
    """
    Normaliza texto y lo divide en tokens (asumiendo el espacio como separación de tokens)
    
    :param texto: Una cadena de caracteres con el texto a normalizar
    
    :return: Una lista de tokens (strings)
    
    """
    
    # AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES

    
    return texto

y ahora vamos a utilizar tu función de normalización para hacer un archivote con todas las frases de ambos `dataframes` ya tratados. Esto hay que hacerlo en forma secuencial, para no tener problemas de memoria. Esta parte la dejo porgramada y lista para usarse.

In [4]:
# Escribe el resultado en un archivo de texto
with open('datos/stackovr_normalizados.txt', 'w', encoding='utf8') as fp:
    for documento in df_post['title'].values:
        fp.write(normaliza_texto(documento) + '\n')
    for documento in df_dialog['text'].values:
        fp.write(normaliza_texto(documento) + '\n')

Y ahora vamos a desarrollar un modelo. Si quisieramos entrenar el modelo en python, muy seguramente la libreta de jupyter se va a resetear por falta de memoria. Es por esta razón que es necesario utilizar la versión compilada de [*FastText*](https://fasttext.cc). En [esta página de documentación](https://fasttext.cc/docs/en/cheatsheet.html) viene los comandos mínimos para las diferentes tareas que pueden hacerse con *FastText*. Revisalas y verifica que puedes construir un modelo de vector de palabras con una dimensión de 100.

Cuando ejecutes el comando tardará bastante tiempo en generar el modelo, pero estará escribiendo información en la salida con el fin que estés seguro que está trabajando.

**Genera un modelo de fasttext con los datos de entrada `datos/stackovr_normalizados.txt`. Guarda el modelo en el archivo `modelos/fstxt_model`**

In [ ]:
![ESCRIBE AQUÍ EL COMANDO]

Y ahora vamos a probar si funciona, cargandolo en la libreta

In [52]:
modelo = gensim.models.FastText.load_fasttext_format('modelos/fstxt_model')

**¿Qué pasa con este modelo cuando proguntamos el típico `king - man + woman?`?, Justifica tu respuesta**

In [4]:
modelo.wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=5)

[('kings', 0.7870528697967529),
 ('wealthy', 0.7674549221992493),
 ('daughterinlaws', 0.7634091973304749),
 ('womans', 0.7623488903045654),
 ('catwoman', 0.7604762315750122)]

**¿Y que pasaría con `python - dictionary + java`?**

**¿Qué otra relación encuentras su resultado particularmente interesante?**

In [5]:
modelo.wv.most_similar(positive=['dictionary', 'java'], negative=['python'], topn=5)

[('hashmap', 0.7939369082450867),
 ('arraylistnamevaluepair', 0.7892711162567139),
 ('arraylisthashmap', 0.7810637950897217),
 ('arraylistcustomobject', 0.7795299291610718),
 ('arrayliststring', 0.776357889175415)]

Por útimo revisemas a que etiqueta se parece más una palabra (si es que existe).

1. **Verifica que cada `tag` se parezca a el mismo. Es una manera trivial de verificar el modelo.**
2. **Verifica cuales son los `tags` que se parecen entre si**
3. **Verifica si expresiones muy tipicas de un lenguaje efectivamente reconoce a ese lenguaje como el más cercano (o de las mas cercanos), por ejemplo `malloc`**


In [140]:
tags_vec = np.zeros((len(tags), modelo.vector_size))
for (i, tag) in enumerate(tags.keys()):
    tags_vec[i,:] = modelo[tag]

palabra = 'java'
sims = modelo.wv.cosine_similarities(modelo[palabra], tags_vec)

print("Similaridad de los 'tags' con la palabra " + palabra)
for (tag, sim) in zip(tags.keys(), sims):
    print("{:15}{}".format(tag, sim))

Similaridad de los 'tags' con la palabra java
swift          0.4261448733027299
c#             0.6097577096650382
r              0.46793525039293277
ruby           0.504294792051381
java           0.9999999737444454
python         0.5606679928235346
javascript     0.5849551802099667
php            0.5286095699599829
vb             0.4789812059497006
c_cpp          0.3565544366319904


## 3. Vectores de frases

Ahora si, ya tenemos un modelo entrenado para nuestro problema particular, y confiamos en él. Ahora lo que necesitamos es desarrollar una función que permita parametrizar una frase y no solo una palabra. Como vimos en clase, la mejor manera de parametrizar una frase es encontrando la codificación para cada palabra que exista en el vocabulario, y luego calculando la media de todos los vectores.

Dado que estamos utilizando *FastText*, es posible encontrar los vectores de palabras que no existan en el vocabulario, por lo que es mejor probar si las palabras tienen codificación o no.

** Desarrolla la función `doc_a_vec`**

Ayuda: si aplicamos 
```python
modelo['palabra otra_palabra tercera_palabra]
```

obtenemos un ndarray de /shape/ `[3, 100]`

y recuerda que buscamos a la salida un ndarray de *shape* `[100, ]`

In [53]:
def doc_a_vec(doc, modelo):
    """
    Convierte un documento en vector utilizando
    el viejo truco de sacar la media de las palabras
    que existen en el vocabulario.
    
    :param doc: Una lista con las palabras del documento
    :param modelo: Un modelo tipo FastText o cualquiero otro modelo de vectores
    
    :return un vector que representa un documento
    
    """
    
    # AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES    
    
    return 

## 4. Generando un discriminador de pregunta técnica vs texto normal con el modelo

Para realizar el modelado, vamos a seleccionar un número de titulos de preguntas a *Stackoverflow* de la misma dimensión que el número de diálogos de cine, con la idea de entrenar un clasificador con el mísmo numero de ejemplos de cada clase y evitar que esté desbalanceado.

In [13]:
num_muestras = df_dialog.shape[0]
positivos = df_post['title'].sample(num_muestras, random_state=10).values
negativos = df_dialog['text'].values

x_texto = np.r_[positivos, negativos]
y = np.r_[np.ones(num_muestras), -1 * np.ones(num_muestras)]

Y ahora vamos a comvertir estos datos a vectores de frases con la funcion que realizaste. Fijate en dos cosas en esta función:

1. Convertimos todas las entradas a `float32`, ya que numpy se basa en `float64` y es mucha la memoria sin que aporte realmente mucho más.

2. Verificamos cuantas frases, o estaban en blanco (i.e. puras palabras de paro) o no pudieron codificarse (i.e. puras palabras OOV que no pudo generalizar *FastText*). Si tenemos suerte, habrá menos del 2% de frases sin clasificar, y la mayoría pertenecen a una sola clase.

**Revisa que sea el caso, y si no lo es, regresa tus pasos para ver que tienes que modificar.**


In [14]:
x = np.zeros((2 * num_muestras, modelo.vector_size), dtype=np.float32)
pos_0, neg_0 = 0, 0
for (i, doc) in enumerate(x_texto):
    docn = normaliza_texto(doc).split()
    try:
        x[i, :] = doc_a_vec(docn, modelo)
    except:
        if y[i] > 0:
            pos_0 += 1
        else:
            neg_0 += 1
        
print("sin poner positivos: {},\tnegativos: {}".format(pos_0, neg_0))

sin poner positivos: 22,	negativos: 2081


**Divide los conjuntos de datos `x`y `y`en conjuntos de aprendizaje y validación (10% de datos de validación), y ajusta un clasificador por regresión logística**

In [16]:
# AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES

x_entrena, x_valida, y_entrena, y_valida = # AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES

clf_tipo = # AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES

# AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Ahora, con una función reporte, vamos a verificar sobre el conjunto de datos de validación si el clasificador 

In [62]:
def reporte(y, y_est, labels=None):
    print("\nPorcentaje de acierto: {}".format(accuracy_score(y, y_est)))
    print("\nPrecisión, recall y f1-score")
    print(classification_report(y, y_est, target_names=labels))    

** Verifica que los datos de validación tienen un error menor al 2%**

In [20]:
y_valida_est = clf_tipo.predict(x_valida)
print("\n\nPara los datos de validación\n" + 40*"=")
reporte(y_valida, y_valida_est)



Para los datos de validación

Porcentaje de acierto: 0.992383696994648

Precisión, recall y f1-score
             precision    recall  f1-score   support

       -1.0       0.99      0.99      0.99     22020
        1.0       0.99      0.99      0.99     21702

avg / total       0.99      0.99      0.99     43722



Por último guardamos el clasificador, por si algo pasa, ya no tenemos que repetir todos estos pasos.

In [21]:
with open("modelos/clf_tipo.pkl", "wb") as fp:
    pickle.dump(clf_tipo, fp)

## 4. Generando un clasificador de tópicos con el modelo

Para este ejercicio, los pasos son casi iguales que para el inciso anterior, solo que con una dificultad mayor, ya que la cantidad de datos de entrenamiento es bestial y el algoritmo de regresion logística tal cual no va a funcionar (puedes verificarlo).

Primero vamos a convertir *todos* los titulos de las preguntas de *Stackoverflow* en vectores. Esto se va a llevar una buena cantidad de minutos (o más dependiendo del número de procesadores y la velocidad de tu equipo). De nuevo, espereríamos que la cantidad de frases sin codificar fueran despreciables respecto al volumen de frases de cada `tag`.

In [23]:
x = np.zeros((df_post.shape[0], modelo.vector_size), dtype=np.float32)
tags_lost = {tag: 0 for tag in tags.keys()}

for (i, doc) in enumerate(df_post['title'].values):
    docn = normaliza_texto(doc).split()
    try:
        x[i, :] = doc_a_vec(docn, modelo)
    except:
        tags_lost[df_post.loc[i, 'tag']] += 1
print("Documentos sin representación densa por etiqueta")
for tag in tags.keys():
    print("Para la etiqueta '{}' hay {} de {}".format(tag, tags_lost[tag], tags[tag]))

Documentos sin representación densa por etiqueta
Para la etiqueta 'php' hay 17 de 321752
Para la etiqueta 'javascript' hay 16 de 375867
Para la etiqueta 'python' hay 10 de 208607
Para la etiqueta 'r' hay 5 de 36359
Para la etiqueta 'java' hay 21 de 383456
Para la etiqueta 'vb' hay 4 de 35044
Para la etiqueta 'c_cpp' hay 46 de 281300
Para la etiqueta 'swift' hay 1 de 34809
Para la etiqueta 'ruby' hay 4 de 99930
Para la etiqueta 'c#' hay 26 de 394451


Y guardamos los datos tratados (comprimiendolos al mismo tiempo) para no tener que volverlos a generar.

In [24]:
np.savez_compressed('datos/vector_x', x=x)

**Entrena un clasificador de tema (lenguaje de programación) con los daots que tenemos, utilizando el 20% de validación.**

Ayuda: Checa la función SGDClassifier (Gradiente estocástico) para grandes volumenes de datos y revisa en que caso es equivalente a una regresión logística y cuando una máquina de vector de soporte. Usa la que consideres mñas conveniente, pero deja explicitamente cual usaste.

In [60]:
x_entrena, x_valida, y_entrena, y_valida = # AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES

clf_tema = # AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES

# AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

**Revisa que el error en los datos de validación no exceda el 25%**

In [64]:
y_valida_est = clf_tema.predict(x_valida)
print("\n\nPara los datos de validación\n" + 40*"=")
reporte(y_valida, y_valida_est)



Para los datos de validación

Porcentaje de acierto: 0.7859525920127096

Precisión, recall y f1-score
             precision    recall  f1-score   support

         c#       0.75      0.78      0.77     78965
      c_cpp       0.77      0.79      0.78     56373
       java       0.82      0.81      0.82     76788
 javascript       0.76      0.83      0.80     74808
        php       0.81      0.79      0.80     64546
     python       0.86      0.83      0.84     41799
          r       0.88      0.76      0.82      7160
       ruby       0.87      0.81      0.84     19967
      swift       0.85      0.38      0.53      6905
         vb       0.02      0.01      0.01      7004

avg / total       0.78      0.79      0.78    434315



Y por último guardamos el modelo

In [65]:
with open("modelos/clf_tema.pkl", "wb") as fp:
    pickle.dump(clf_tema, fp)

## 5. Construyendo un recomendador a partir del modelo

Ahora, si podemos detectar si la frase parece una pregunta de *Stackoverflow*, y sobre que tema es la pregunta, deberíamos ser capaces de recomendar checar alguna de las preguntas existentes (o varias en orden de importancia, pero por el momento nos quedamos en una).

Par eso, pues simplemente vamos a comparar la frase (en forma de vector), con las frases existentes utilizando la *similaridad coseno*. Primero vamos a obtener los indices de llas frases de cada una de las etiquetas, y vamos a guardar ese vector de indices en un diccionario:

In [66]:
y = df_post.loc[:,'tag']
indices = {tag: np.where( y == tag)[0]
           for tag in tags.keys()
          }

Ahora hay que hacer el recomendador.

**Desarrolla la función que encuentre el indice del vector de x mas similar a pregunta_vec**

Ayuda: revisa modelo.wv.cosine_similarities. Ten cuidado, si algun vector es de puros 0, entonces te puedes encontrar un NaN entre las similaridades.

In [153]:
def recomienda_pregunta(modelo, pregunta_vec, x):
    """
    Encuentra el índice del vector de x con mayor simiaridad a pregunta_vec
    utilizando la similaridad coseno.
    
    param modelo: Un modelo tipo fastext de gensim
    param pregunta_vec: Un ndarray [100,] con la codificación de la frase
    param x: Un ndarray [n_ejemplos, 100] con la codificación de los ejemplos.
    """
    
    # AGREGA AQUÍ TODO EL CÓDIGO QUE NECESITES
    
    

Para probar tu función, el ejercicio de abajo debería salir:

```
post_id                4551876
title      Java Multithreading
tag                       java
Name: 295499, dtype: object
```

In [154]:
pregunta = "Multithreading in Java"

pregunta_vec = doc_a_vec(normaliza_texto(pregunta).split(), modelo)

ind = indices['java']
i = recomienda_pregunta(modelo, pregunta_vec, x[ind, :])
posicion = indices['java'][i] 

df_post.loc[posicion,:]

post_id                4551876
title      Java Multithreading
tag                       java
Name: 295499, dtype: object

## 6. Poniendo todo junto

Por último vamos a poner todo junto como si fuera parte de un sistema de asesoría automática (de hecho es un sistema de recomendación basado en lenguaje natural).

Lo primero, vamos a cargar todo lo que necesitamos que ya obtuvimos anteriormente: (si volviste a empezar de cero, la primer celd y las que tienen funciones auxiliares hay que volver a ejecutarlas).

In [69]:
x = np.load('datos/vector_x.npz')['x']

modelo = gensim.models.FastText.load_fasttext_format('modelos/fstxt_model')

with open("modelos/clf_tipo.pkl", "rb") as fp:
    clf_tipo = pickle.load(fp)

with open("modelos/clf_tema.pkl", "rb") as fp:
    clf_tema = pickle.load(fp)


Y ahora juntemos todo lo que se desarrollo para hacer un recomendador muy sencillito.

In [167]:
def recomendador(pregunta, modelo, clf_tipo, clf_tema, df_post):
    """
    Recomienda una respuesta posible a una posible pregunta sobre
    programación, basado en un modelo de semantica distrbuida 
    
    :param pregunta: Una cadena de caracteres con una frase
    :param modelo: Un modelo tipo fasttext
    :param clf_tipo: Clasificado de tipo, > 0, nos interesa el documento
    :param clf_tema: Clasificador seleccionador del tema
    :param df_post: El dataframe de pandas de las preguntas, tal cual.
    """
    pregunta_vec = doc_a_vec(normaliza_texto(pregunta).split(), modelo)
    
    if clf_tipo.predict(pregunta_vec.reshape(1, -1)) < 0:
        return "Esta es una pregunta que no nos interesa por el momento, y me da flojera contestarte.", None
    
    tag = clf_tema.predict(pregunta_vec.reshape(1, -1))[0]
    
    pregunta_vec.ravel()
    i = recomienda_pregunta(modelo, pregunta_vec, x[indices[tag],:])
    ind = indices[tag][i]
    
    return ("Hay una pregunta parecida en Stackoverflow sobre el lenguaje {} y dice: \
             \n <<{}>>".format(tag, df_post.loc[ind, 'title']),
            df_post.loc[ind,'post_id'])


Listo, solo falt porbar nuestro recomendador, a ver que ten bien funciona. Yo puse 10 frases para probarlo. 

**Agrega otras 10 frases para probarlo. Debe al menos probarse con una pregunta cada uno de los `tags`. **

In [169]:
# algunas frases para probar:
f1 = "How to use regular expresions"
f2 = "How to delete rows in pandas?"
f3 = "Multithreading in Java"
f4 = "pandas dataframes"
f5 = "What is Natural Language Processing?"
f6 = "What is Artificial Intelligence?"
f7 = "How to apply word2vec to sentiment analysis"
f8 = "I want to vrite a program for Iphone 8"
f9 = "Program with databases in linux"
f10 = "Program with databases in MacOS"


cadena, idq = recomendador(f1, modelo, clf_tipo, clf_tema, df_post)

print(cadena + '\n')

if idq is not None:
    pagina = "https://stackoverflow.com/questions/" + str(idq)
    #display(HTML(url=pagina))
    print("Y se puede consultar en " + pagina)

Hay una pregunta parecida en Stackoverflow sobre el lenguaje javascript y dice:              
 <<How do I include - and ' in this regular expressions?>>

Y se puede consultar en https://stackoverflow.com/questions/3659848


**Prueba tu sistema de recomandación y determina cuando funciona bien, cuando mal y que podríamos hacer para mejorar. Describe todo esto y tus conclusiones aquí mismo abajo de las instrucciones**.

ESCRIBE TUS CON CONCLUSIONES AQUI